In [23]:
!pip install flask
!pip install pyspark


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [24]:
from flask import Flask, request, jsonify
from pyspark.sql import SparkSession
import os

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressionModel

# Create a new SparkSession
spark = SparkSession.builder.appName("FlaskApp").getOrCreate()
print("Done creating Spark Session!")
model = None

def loadModel(modelPath=None):
    global model
    path = "savedModel/randomForestpredictionmodel/"
    if modelPath:
        path = modelPath
    model = RandomForestRegressionModel.load(path)
    print("Done loading Model!")

# load model initially
loadModel()

# Prediction Function
def predict_volume(input_data):
    global model

    print(os.curdir)
    # Assemble the features into a vector column
    assembler = VectorAssembler(inputCols=['vol_moving_avg', 'adj_close_rolling_med'], outputCol="features")
    input_data = assembler.transform(input_data)

    # Make predictions on the new data
    predictions = model.transform(input_data)

    # Select relevant columns and return the DataFrame
    output = predictions.select('prediction')
    return output

app = Flask(__name__)

@app.route('/')
@app.route('/health')
def health():
    return jsonify({"message":'App is healthy!'})

@app.route('/predict', methods=['GET'])
def predict():
    # Get the values of the 'vol_moving_avg' and 'adj_close_rolling_med' query parameters
    vol_moving_avg = float(request.args.get('vol_moving_avg'))
    adj_close_rolling_med = float(request.args.get('adj_close_rolling_med'))

    input_data = spark.createDataFrame([(vol_moving_avg, adj_close_rolling_med)], ['vol_moving_avg', 'adj_close_rolling_med'])

    predictions = predict_volume(input_data)
    # Convert the predictions to a JSON response
    output = {'predictions': predictions.select('prediction').collect()}
    return jsonify(output)

@app.route('/loadNewModel')
def loadModelApi():
    request.args.post('modelPath')
    loadModel()
    return jsonify({"message":'App is healthy!'})

Done creating Spark Session!
Done loading Model!


In [25]:
app.run(host="0.0.0.0",port=8000)

 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.16.5.4:8000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.16.5.4:8000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit


.


127.0.0.1 - - [04/May/2023 20:54:24] "GET /predict?vol_moving_avg=2.22&adj_close_rolling_med=0.23 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2023 20:54:24] "GET /predict?vol_moving_avg=2.22&adj_close_rolling_med=0.23 HTTP/1.1" 200 -


...

127.0.0.1 - - [04/May/2023 20:55:27] "GET /predict?vol_moving_avg=2.22&adj_close_rolling_med=0.23 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2023 20:55:27] "GET /predict?vol_moving_avg=2.22&adj_close_rolling_med=0.23 HTTP/1.1" 200 -


.


127.0.0.1 - - [04/May/2023 20:55:27] "GET /predict?vol_moving_avg=3.89&adj_close_rolling_med=2 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2023 20:55:27] "GET /predict?vol_moving_avg=3.89&adj_close_rolling_med=2 HTTP/1.1" 200 -


.


127.0.0.1 - - [04/May/2023 20:55:28] "GET /predict?vol_moving_avg=1.9&adj_close_rolling_med=9 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2023 20:55:28] "GET /predict?vol_moving_avg=1.9&adj_close_rolling_med=9 HTTP/1.1" 200 -
